In [ ]:
!pip install skipthoughts

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import sys

In [ ]:
from skipthoughts import BiSkip

my_file = open("dictionary.txt", "r")
data = my_file.read()
data_into_list_temp = data.replace('\n', ' ').split(" ")
data_into_list = list(set(data_into_list_temp))

dir_st = 'data/skip-thoughts'
vocab = data_into_list
biskip = BiSkip(dir_st, vocab)

In [ ]:
my_dict = {}
for i in range(len(vocab)):
  my_dict[vocab[i]] = i+1

In [ ]:
my_dict['<eos>']

In [ ]:
import pickle

In [ ]:
pickle.dump(my_dict, open('my_dict.pkl', 'wb'))

In [ ]:
pickle.dump(biskip, open('biskip.pkl', 'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
my_dict = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/my_dict.pkl', 'rb'))
biskip = pickle.load(open('gdrive/MyDrive/CS626/Project/Data/biskip.pkl', 'rb'))

In [ ]:
def neural_features(dataset_location):
  dataset = pd.read_csv(dataset_location)
  headlines = dataset['Headline']
  bodies = dataset['Body']
  MAX_HEADLINE_COUNT = dataset['Headline Word Count'].max()
  MAX_BODY_COUNT = dataset['Body Word Count'].max()
  headlines_to_ids = np.zeros((len(headlines),MAX_HEADLINE_COUNT+1))
  bodies_to_ids = np.zeros((len(bodies),MAX_BODY_COUNT+1))
  headlines_encodings = np.zeros((len(headlines),2400))
  bodies_encodings = np.zeros((len(bodies),2400))
  for i in range(len(headlines)):
    headline = headlines[i].split()
    headline.append('<eos>')
    body = bodies[i].split()
    body.append('<eos>')
    j=0
    for word in headline:
      try:
        headlines_to_ids[i][j] = my_dict[word]
      except KeyError:
        pass
      j+=1
    j=0
    for word in body:
      try:
        bodies_to_ids[i][j] = my_dict[word]
      except KeyError:
        pass
      j+=1
  last_temp = len(headlines) - len(headlines)%50
  for i in range(0,len(headlines),50):
    print(i)
    input1 = Variable(torch.LongTensor(headlines_to_ids[i:i+50]))
    input2 = Variable(torch.LongTensor(bodies_to_ids[i:i+50]))
    headline_output = biskip(input1).detach().numpy()
    body_output = biskip(input2).detach().numpy()
    headlines_encodings[i:i+50] = headline_output[0:50]
    bodies_encodings[i:i+50] = body_output[0:50]
  if(last_temp != len(headlines)):
    print(last_temp)
    input1 = Variable(torch.LongTensor(headlines_to_ids[last_temp:]))
    input2 = Variable(torch.LongTensor(bodies_to_ids[last_temp:]))
    headline_output = biskip(input1).detach().numpy()
    body_output = biskip(input2).detach().numpy()
    headlines_encodings[last_temp:] = headline_output[:]
    bodies_encodings[last_temp:] = body_output[:]

  feat1 = np.zeros((len(headlines),2400))
  feat2 = np.zeros((len(headlines),2400))
  i = 0
  for h_vector,b_vector in zip(headlines_encodings,bodies_encodings):
    feat1[i] = np.multiply(h_vector,b_vector)
    feat2[i] = np.absolute(h_vector-b_vector)
    i+=1

  final_neural_features = np.concatenate((feat1,feat2),axis = 1)
  return final_neural_features


In [ ]:
neural_features_train = neural_features('gdrive/MyDrive/CS626/Project/Data/train_Set.csv')  

In [ ]:
np.save(arr=neural_features_train,file='gdrive/MyDrive/CS626/Project/Data/train_neural_features.npy')

In [ ]:
neural_features_test = neural_features('gdrive/MyDrive/CS626/Project/Data/test_Set.csv')

In [ ]:
np.save(arr=neural_features_test,file='gdrive/MyDrive/CS626/Project/Data/test_neural_features.npy')

In [ ]:
print(neural_features_test[25402][100:150])